In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import balanced_accuracy_score
# from sklearn.preprocessing import LabelEncoder

import catboost as cb
from catboost import CatBoostClassifier, Pool

import xgboost as xgb
from xgboost import XGBClassifier, DMatrix

import matplotlib.pyplot as plt
import seaborn as sns
import phik
from phik.report import plot_correlation_matrix
from phik import report

import warnings
warnings.filterwarnings('ignore')

C:\Users\koles\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
train = pd.read_csv('https://www.dropbox.com/scl/fi/9hb4r3uce0mqz8fkpja17/text_classification_train.csv?rlkey=w42y98wa401gelzou08pp582k&dl=1')

test = pd.read_csv('https://www.dropbox.com/scl/fi/7z7rsy14amjeugf166i1t/text_classification_test.csv?rlkey=z53jgwhijd6bpvk7n8n2munwb&dl=1')

# Baseline model

In [13]:
train.shape, test.shape

((7500, 2618), (2500, 2617))

In [15]:
train.head()

,category,text,ruBert-base_text_feature_0,ruBert-base_text_feature_1,ruBert-base_text_feature_2,ruBert-base_text_feature_3,ruBert-base_text_feature_4,ruBert-base_text_feature_5,ruBert-base_text_feature_6,ruBert-base_text_feature_7,...,labse_text_feature_758,labse_text_feature_759,labse_text_feature_760,labse_text_feature_761,labse_text_feature_762,labse_text_feature_763,labse_text_feature_764,labse_text_feature_765,labse_text_feature_766,labse_text_feature_767
0,extreme,Ледник Пасторури это цирковой ледник расположе...,0.272156,0.155383,0.060285,0.363159,-0.140391,0.507753,-0.226326,0.431878,...,-0.045795,-0.027475,0.030528,-0.052218,0.042459,-0.012714,-0.055370,-0.012433,-0.016283,-0.006994
1,martial_arts,Главные участники предстоящего Betokenoid 274 ...,0.439223,0.343683,0.093642,0.245294,0.089770,0.424717,-0.071487,0.185970,...,-0.022664,0.042258,-0.027394,-0.033566,0.016021,-0.022054,-0.040366,0.007392,-0.029070,-0.011284
2,extreme,Ttokenoid Btokenoid – карта с которой можно не...,-0.040338,0.058095,-0.091063,0.296028,-0.137103,0.931456,-0.169060,0.131503,...,-0.009464,0.011548,-0.046034,0.024588,-0.013670,-0.047028,0.009395,-0.000488,-0.060260,0.006563
3,autosport,В Сильверстоуне произошли крупные обновления а...,0.444181,0.218742,0.247859,0.234885,0.006668,0.407703,-0.115768,0.433781,...,-0.045474,-0.043454,-0.000961,-0.012203,-0.047922,-0.054657,-0.053768,0.018481,-0.039148,-0.038874
4,extreme,На протяжении более чем 30 лет Вестсайд являет...,-0.126253,-0.115856,0.131131,0.052595,0.060591,0.420976,0.090776,0.246287,...,0.004818,-0.008138,-0.013958,-0.038482,-0.002943,-0.035970,-0.010830,-0.005132,-0.047990,-0.005869


In [4]:
X = train.drop(['category', 'text'], axis=1)
y = train['category'].astype('category')
num_classes = y.nunique()
cat_features = ['category']

In [5]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42)


In [9]:
train_pool = cb.Pool(X_train, y_train)
test_pool = cb.Pool(X_test, y_test)

model = cb.CatBoostClassifier(
    iterations=1000,
    depth=6,
    loss_function='MultiClass',
    classes_count=num_classes,
    verbose=100,
    task_type="GPU",
    devices='0'
)

In [48]:
model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=100)

Learning rate set to 0.126016
0:	learn: 2.4743254	test: 2.4877300	best: 2.4877300 (0)	total: 388ms	remaining: 6m 27s
100:	learn: 0.7364787	test: 1.2648581	best: 1.2648581 (100)	total: 27.9s	remaining: 4m 8s
200:	learn: 0.4491232	test: 1.0993957	best: 1.0993957 (200)	total: 52.8s	remaining: 3m 29s
300:	learn: 0.3244822	test: 1.0361701	best: 1.0361701 (300)	total: 1m 15s	remaining: 2m 55s
400:	learn: 0.2499024	test: 0.9976979	best: 0.9975969 (399)	total: 1m 38s	remaining: 2m 26s
500:	learn: 0.2021835	test: 0.9691122	best: 0.9691122 (500)	total: 1m 59s	remaining: 1m 59s
600:	learn: 0.1646249	test: 0.9439510	best: 0.9439510 (600)	total: 2m 21s	remaining: 1m 34s
700:	learn: 0.1375408	test: 0.9262325	best: 0.9262325 (700)	total: 2m 43s	remaining: 1m 9s
800:	learn: 0.1152857	test: 0.9096509	best: 0.9096003 (799)	total: 3m 5s	remaining: 46.1s
900:	learn: 0.0984762	test: 0.8976452	best: 0.8976452 (900)	total: 3m 27s	remaining: 22.8s
999:	learn: 0.0857581	test: 0.8881405	best: 0.8881405 (999)	to

In [49]:
y_pred = model.predict(X_test)

# Преобразуйте числовые предсказания обратно в строковые метки
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)

# Оцените точность модели
accuracy = balanced_accuracy_score(y_test_labels, y_pred_labels)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7239302608475016


In [51]:
model.save_model('models/catboost_baseline_model.cbm')

In [10]:
model = CatBoostClassifier()
model.load_model('models/catboost_baseline_model.cbm')

In [11]:
feature_importances = model.get_feature_importance(data=train_pool, type=cb.EFstrType.PredictionValuesChange)

feature_importances_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances}).sort_values(by='Importance', ascending=False)

feature_importances_df

,Feature,Importance
641,ruBert-base_text_feature_641,1.298949
2036,labse_text_feature_188,1.298233
113,ruBert-base_text_feature_113,1.130945
86,ruBert-base_text_feature_86,0.997096
423,ruBert-base_text_feature_423,0.986294
...,...,...
1660,rubert-base-cased-conversational_text_feature_580,0.000000
1662,rubert-base-cased-conversational_text_feature_582,0.000000
1663,rubert-base-cased-conversational_text_feature_583,0.000000
1664,rubert-base-cased-conversational_text_feature_584,0.000000


отбросим фичи с важностью < 0.03

In [29]:
drop_feature_importances_df = feature_importances_df[feature_importances_df['Importance'] < 0.03]
drop_feature_importances_df.shape

(1953, 2)

In [30]:
train_cleaned = train.drop(drop_feature_importances_df['Feature'].to_list(), axis=1)
test_cleaned = test.drop(drop_feature_importances_df['Feature'].to_list(), axis=1)

train_cleaned.to_csv('data/train_cleaned.csv')
test_cleaned.to_csv('data/test_cleaned.csv')

В дальнейшем:
1. затюнить в Оптуне
2. Обучить xgb (dart) и затюнить
3. rf и затюнить
4. Заблендить резы

X

In [31]:
X = train_cleaned.drop(['category', 'text'], axis=1)
y = train['category']

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42)

In [32]:
train_pool = cb.Pool(X_train, y_train)
test_pool = cb.Pool(X_test, y_test)

model = cb.CatBoostClassifier(
    iterations=1000,
    depth=6,
    loss_function='MultiClass',
    classes_count=num_classes,
    verbose=100,
    task_type="GPU",
    devices='0'
)

In [33]:
model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=100)

Learning rate set to 0.126016
0:	learn: 2.4743252	test: 2.4877301	best: 2.4877301 (0)	total: 116ms	remaining: 1m 55s
100:	learn: 0.7347393	test: 1.2751903	best: 1.2751903 (100)	total: 8.9s	remaining: 1m 19s
200:	learn: 0.4261100	test: 1.1128776	best: 1.1128776 (200)	total: 16.7s	remaining: 1m 6s
300:	learn: 0.2996103	test: 1.0380178	best: 1.0380178 (300)	total: 23.9s	remaining: 55.6s
400:	learn: 0.2266842	test: 0.9978951	best: 0.9978951 (400)	total: 30.9s	remaining: 46.2s
500:	learn: 0.1830882	test: 0.9734650	best: 0.9733679 (498)	total: 37.8s	remaining: 37.6s
600:	learn: 0.1485627	test: 0.9498780	best: 0.9498780 (600)	total: 44.6s	remaining: 29.6s
700:	learn: 0.1226655	test: 0.9295333	best: 0.9295333 (700)	total: 51.5s	remaining: 22s
800:	learn: 0.1036656	test: 0.9174787	best: 0.9174306 (799)	total: 58.4s	remaining: 14.5s
900:	learn: 0.0887874	test: 0.9071838	best: 0.9071838 (900)	total: 1m 5s	remaining: 7.18s
999:	learn: 0.0770977	test: 0.9004686	best: 0.9003856 (998)	total: 1m 12s	r

In [ ]:
y_pred = model.predict(X_test)

# Преобразуйте числовые предсказания обратно в строковые метки
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)

# Оцените точность модели
accuracy = balanced_accuracy_score(y_test_labels, y_pred_labels)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7285750611194073


In [40]:
model.save_model('models/catboost_cleaned_model.cbm')

точность немного выросла, и обучение модели сократилось в 3 раза

In [35]:
import optuna

In [37]:
# def objective(trial):
#     params = {
#         'iterations': 1000,
#         'depth': trial.suggest_int('depth', 4, 10),
#         'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
#         'border_count': trial.suggest_int('border_count', 32, 255),
#         'random_strength': trial.suggest_loguniform('random_strength', 1e-9, 10.0),
#         'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 1.0),
#         'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#         'od_wait': trial.suggest_int('od_wait', 10, 50)
#     }

#     model = CatBoostClassifier(**params, verbose=0, task_type="GPU", devices='0')
#     model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=100, verbose=0)
#     preds = model.predict(X_test)
#     accuracy = balanced_accuracy_score(y_test, preds)

#     # Pruning
#     trial.report(accuracy, step=model.tree_count_)
#     if trial.should_prune():
#         raise optuna.exceptions.TrialPruned()

#     return accuracy

# # Создание исследования Optuna с pruning
# study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
# study.optimize(objective, n_trials=50)

# # Вывод лучших гиперпараметров
# print("Best trial: score {},\nparams {}".format(study.best_trial.value, study.best_trial.params))

XGB (dart mode)

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'multi:softprob',
    'num_class': num_classes,
    'eval_metric': 'mlogloss',
    'booster': 'dart',
    'rate_drop': 0.1,
    'skip_drop': 0.5,
    'tree_method': 'gpu_hist', 
    'gpu_id': 0  
}

num_round = 1000
bst = xgb.train(params, dtrain, num_round)

In [42]:
y_pred_proba = bst.predict(dtest)
y_pred = y_pred_proba.argmax(axis=1)
accuracy = balanced_accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 66.86%


In [43]:
bst.save_model('models/xgboost_dart_model.json')

Random Forest

In [54]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=1000, random_state=42)

model.fit(X_train, y_train)


RandomForestClassifier(n_estimators=1000, random_state=42)

In [55]:
y_pred = model.predict(X_test)
accuracy = balanced_accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 67.61%


XGB - стандартный бустер

In [50]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'multi:softprob',
    'num_class': num_classes,
    'tree_method': 'gpu_hist',  
    'gpu_id': 0,  
    'max_depth': 6,
    'eta': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

num_round = 1000
bst = xgb.train(params, dtrain, num_round)

In [51]:
y_pred_proba = bst.predict(dtest)
y_pred = y_pred_proba.argmax(axis=1)
accuracy = balanced_accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 73.39%


In [52]:
bst.save_model('models/xgboost_gbtree_model.json')

TF-IDF + SVC

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(train['text'], y_encoded, random_state=42, test_size=0.2)

In [89]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [90]:
svm = SVC(kernel='rbf')
svm.fit(X_train_tfidf, y_train)

SVC()

In [63]:
y_pred = svm.predict(X_test_tfidf)
accuracy = balanced_accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7375278288549633


In [65]:
import pickle

with open('models/svm_model.pkl', 'wb') as model_file:
    pickle.dump(svm, model_file)

with open('models/tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

Собираем Блендинг

In [ ]:
cb_model = CatBoostClassifier()
cb_model.load_model('models/catboost_cleaned_model.cbm')

xgb_model = XGBClassifier()
xgb_model.load_model('models/xgboost_gbtree_model.json')

with open('models/svm_model.pkl', 'rb') as model_file:
    svm_model = pickle.load(model_file)

with open('models/tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    vectorizer_loaded = pickle.load(vectorizer_file)


In [68]:
df_test = pd.read_csv('data/test_cleaned.csv')

In [71]:
cb_X = df_test.drop(['Unnamed: 0', 'text'], axis=1)
y_pred = cb_model.predict(cb_X)

In [78]:
cb_y_pred_labels = label_encoder.inverse_transform(y_pred)

In [80]:
xgb_X = df_test.drop(['Unnamed: 0', 'text'], axis=1)
xgb_model.n_classes_ = num_classes
y_pred = xgb_model.predict(xgb_X)
xgb_y_pred_labels = label_encoder.inverse_transform(y_pred)

In [99]:
sv_X = df_test['text']
sv_X = vectorizer.transform(sv_X)

In [100]:
y_pred = svm.predict(sv_X)
svm_y_pred_labels = label_encoder.inverse_transform(y_pred)

In [102]:
predictions = pd.DataFrame({'catboost': cb_y_pred_labels, 'xgb': xgb_y_pred_labels, 'svm': svm_y_pred_labels})

In [104]:
predictions

,catboost,xgb,svm
0,extreme,extreme,extreme
1,extreme,martial_arts,boardgames
2,esport,esport,esport
3,athletics,athletics,athletics
4,autosport,autosport,autosport
...,...,...,...
2495,boardgames,boardgames,boardgames
2496,esport,esport,martial_arts
2497,hockey,hockey,autosport
2498,autosport,autosport,esport


In [105]:
predictions['category'] = predictions[['catboost', 'xgb', 'svm']].mode(axis=1)[0]

In [107]:
predictions['category'].to_csv('submissions/blend.csv', index=False)